In [1]:
#Install dependencies if not already installed
!pip install fastestimator ~/repos/fledge

Processing /home/itrushkin/repos/fledge


  Created wheel for fledge: filename=fledge-0.20-py3-none-any.whl size=168427 sha256=7a4d020d89d4aff2d75db6e1673eb87b8fad0a477d82a5fdaf46b4416a272c96
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrbbndpd/wheels/3a/30/9d/a88101effd240e4a8721301bebf95a245eaf4904b55b7ae530
Successfully built fledge
  Attempting uninstall: fledge
    Found existing installation: fledge 0.20
    Uninstalling fledge-0.20:
      Successfully uninstalled fledge-0.20
You should consider upgrading via the '/home/itrushkin/.virtualenvs/fledge/bin/python -m pip install --upgrade pip' command.


In [2]:
import fastestimator as fe
from fastestimator.architecture.pytorch import LeNet
from fastestimator.backend import to_tensor, argmax
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop import Average
from fastestimator.op.tensorop.gradient import Watch, FGSM
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util import ImgData, to_number
from fledge.native.fastestimator import FederatedFastEstimator
from fastestimator.dataset.data import cifar10
from fastestimator.trace.adapt import LRScheduler
from fastestimator.op.numpyop.univariate import Normalize, ChannelTranspose

batch_size=128

train_data, eval_data = cifar10.load_data()
test_data = eval_data.split(0.5)

pipeline = fe.Pipeline(train_data=train_data,
                     eval_data=eval_data,
                     test_data=test_data,
                     batch_size=batch_size,
                     ops=[Normalize(inputs="x", outputs="x",
                             mean=(0.4914, 0.4822, 0.4465), 
                             std=(0.2471, 0.2435, 0.2616)),
                          ChannelTranspose(inputs="x", outputs="x")])
model = fe.build(model_fn=lambda: LeNet(input_shape=(3, 32, 32)), \
                         optimizer_fn="adam", model_name="adv_model")

network = fe.Network(ops=[
                Watch(inputs="x"),
                ModelOp(model=model, inputs="x", outputs="y_pred"),
                CrossEntropy(inputs=("y_pred", "y"), outputs="base_ce"),
                FGSM(data="x", loss="base_ce", outputs="x_adverse", epsilon=0.04),
                ModelOp(model=model, inputs="x_adverse", outputs="y_pred_adv"),
                CrossEntropy(inputs=("y_pred_adv", "y"), outputs="adv_ce"),
                Average(inputs=("base_ce", "adv_ce"), outputs="avg_ce"),
                UpdateOp(model=model, loss_name="avg_ce")
            ])
def lr_schedule(epoch):
    lr = 1 / (100 * epoch)
    return lr

estimator = fe.Estimator(pipeline=pipeline,
                         network=network,
                         epochs=1,
                         traces=[Accuracy(true_key="y", pred_key="y_pred", output_name="clean_accuracy"),
                                Accuracy(true_key="y", pred_key="y_pred_adv", output_name="adversarial_accuracy"),
                                LRScheduler(model=model, lr_fn=lr_schedule)],
                         max_train_steps_per_epoch=None,
                         max_eval_steps_per_epoch=None,
                         monitor_names=["base_ce", "adv_ce"],
                         log_steps=1000)
fledge_estimator = FederatedFastEstimator(estimator, rounds=10)
model=fledge_estimator.fit()

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── tfmnist_inmemory.py
├── agg_to_col_two_signed_cert.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── network.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── aggregator.yaml
│   │   ├── task_runner.yaml
│   │   └── data_loader.yaml
│   └── plan.yaml
├── requirements.txt
├── logs
├── agg_to_col_one_signed_cert.zip
├── cert
│   ├── config
│   │   ├── client.conf
│   │   ├── signing-ca.conf
│   │   ├── server.conf
│   │   └── root-ca.conf
│   ├── client
│   │   ├── col_one.crt
│   │   ├── col_two.csr
│   │   ├── col_two.crt
│   │   ├── col_two.key
│   │   ├── col_one.csr
│   │   └── col_one.ke

/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:172: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


FastEstimator-Test: step: None; epoch: 1; clean_accuracy: 0.1056; adversarial_accuracy: 0.0152; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 2.3219676; avg_ce: 2.312725; base_ce: 2.3034825; adv_model_lr: 0.01; 
FastEstimator-Train: step: 98; epoch: 1; epoch_time: 2.9 sec; 
FastEstimator-Eval: step: 98; epoch: 1; adv_ce: 1.9664894; avg_ce: 1.8881109; base_ce: 1.8097327; clean_accuracy: 0.3144; adversarial_accuracy: 0.2464; 
FastEstimator-Finish: step: 98; total_time: 4.87 sec; adv_model_lr: 0.01; 


FastEstimator-Test: step: 98; epoch: 2; clean_accuracy: 0.3088; adversarial_accuracy: 0.2432; 


...

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 1.5275463; avg_ce: 1.3520498; base_ce: 1.1765535; adv_model_lr: 0.001; 
FastEstimator-Train: step: 98; epoch: 10; epoch_time: 2.94 sec; 
FastEstimator-Eval: step: 98; epoch: 10; adv_ce: 1.6551527; avg_ce: 1.4705645; base_ce: 1.2859762; clean_accuracy: 0.5288; adversarial_accuracy: 0.3648; 
FastEstimator-Finish: step: 98; total_time: 5.12 sec; adv_model_lr: 0.001; 


FastEstimator-Test: step: 98; epoch: 11; clean_accuracy: 0.5128; adversarial_accuracy: 0.3528; 


FastEstimator-Test: step: 98; epoch: 10; clean_accuracy: 0.524; adversarial_accuracy: 0.3568; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 1.4835212; avg_ce: 1.3091288; base_ce: 1.1347362; adv_model_lr: 0.001; 
FastEstimator-Train: step: 98; epoch: 10; epoch_time: 2.89 sec; 
FastEstimator-Eval: step: 98; epoch: 10; adv_ce: 1.618832; avg_ce: 1.4397892; base_ce: 1.2607462; clean_accuracy: 0.5536; adversarial_accuracy: 0.392; 
FastEstimator-Finish: step: 98; total_time: 5.08 sec; adv_model_lr: 0.001; 


FastEstimator-Test: step: 98; epoch: 11; clean_accuracy: 0.5344; adversarial_accuracy: 0.3608; 


In [3]:
model.state_dict()

OrderedDict([('module.conv1.weight',
              tensor([[[[ 1.0226e-01, -1.8061e-02,  6.2101e-02],
                        [ 1.1535e-01,  1.8759e-01, -3.7415e-02],
                        [ 3.6231e-02,  8.9576e-02,  1.1290e-01]],
              
                       [[ 8.4680e-02, -1.9130e-01,  3.6085e-02],
                        [ 5.8601e-02, -2.3066e-02, -4.5776e-02],
                        [-6.2782e-02,  5.1348e-02,  1.1603e-01]],
              
                       [[ 8.0218e-02, -4.9240e-02, -1.1576e-01],
                        [ 1.9658e-02, -8.7544e-02,  7.7558e-02],
                        [-1.5911e-01,  7.0302e-02,  8.0136e-02]]],
              
              
                      [[[-2.3775e-01, -2.2354e-01, -2.6299e-01],
                        [ 1.2773e-01,  6.9567e-02,  4.9831e-02],
                        [ 2.0147e-01,  1.2419e-01,  1.9911e-01]],
              
                       [[-2.0929e-02, -1.3758e-01, -8.1094e-03],
                        [-8.0863e-02, 